In [1]:
class PIDController():

    AUTO_MODE = 1
    OFF_MODE = 0

    def __init__(self, pgain=1, itime=1, dtime=0, auto_max=100,
                 auto_min=0, do_ifactor=False, anti_windup_backcalc=True):

        self.auto_min = auto_min
        self.auto_max = auto_max
        self.pgain = pgain
        self.itime = itime
        self.oneoveritime = 1 / self.itime
        self.dtime = dtime
        self.dif = do_ifactor
        self.awb = anti_windup_backcalc

        self.accumulated_error = 0
        self.bump = 0
        self.last_output = 0
        self.last_error = 0
        self.last_pv = 0

    def off_to_auto(self, pv, sp):
        """
        Calculate bump for off-to-auto transfer
        :param pv: current process temp to use for bumpless xfer calculation
        """
        self.man_to_auto(pv, sp, 0)

    def man_to_auto(self, pv, sp, op):
        err = sp - pv
        uk0 = self.pgain * err
        self.bump = op - uk0
        
    set_bump = man_to_auto

    def reset(self):
        self.accumulated_error = 0
        self.last_pv = 0
        self.last_error = 0        
    
    def step(self, pv, sp, dt=1):
        err = sp - pv
        dpv = pv - self.last_pv
        ierr = (err + self.last_error) / 2 * dt
        
        if self.dif:
            ierr *= 1 / (1 + err * err / 250)  # Labview's ifactor thingy
        
        # bump (aka controller bias) isn't normally
        # included in Up, but no one else reads my 
        # code :)
        Up = self.bump + self.pgain * err
        Ui = self.oneoveritime * (ierr + self.accumulated_error) * self.pgain
        Ud = self.dtime * dpv * self.pgain
        Uk = Up + Ui + Ud
        
        # XXX debugging
        self.Ui = Ui * self.pgain
        self.Uk = Uk
        self.Ud = Ud * self.pgain
        self.Up = Up
        
        # Coercion & back calculation      
        back_calc = False
        if Uk > self.auto_max:
            Uk = self.auto_max
            back_calc = True
            am = self.auto_max
        elif Uk < self.auto_min:
            Uk = self.auto_min
            back_calc = True
            am = self.auto_min
            
        if back_calc and self.awb:
            ierr = self.itime / self.pgain * (am - Up - Ud) - self.accumulated_error
            
        self.accumulated_error += ierr
        self.last_output = Uk
        self.last_pv = pv
        self.last_error = err
        self.last_ierr = ierr
        
        return Uk

    def __repr__(self):
        return "Output: %.2f Pgain: %.1f Itime: %.2f AccumError: %.4f" % (self.last_output,
                                                                          self.pgain,
                                                                          self.itime,
                                                                          self.accumulated_error)
    __str__ = __repr__
    

In [2]:
def assert_equal(a,b):
    assert a == b, (a, b)
def test_trap_integration1():
    data = [
        (0, 1, .5),
        (0, 2, 2),
        (0, 3, 4.5)
    ]
    p = PIDController(1,1)
    for pv, sp, op in data:
        p.off_to_auto(pv, sp)
        res_op = p.step(pv, sp)
        assert_equal(p.last_error, sp)
        assert_equal(p.accumulated_error, op)
        assert_equal(res_op, op)
test_trap_integration1()

def check_Ui_backcalc(it, pg, Up, Ud, ierr, accum, val):
    Ui2 = it * (ierr + accum) * pg
    Uk2 = Up + Ud + Ui
    Uk2 = round(Uk2, 8)
    am = round(val, 8)
    assert Uk2 == am, (Up, Ud, Ui, Uk2, val)

In [3]:
def m2s(m):
    return m*60
def s2m(s):
    return s/60
def h2s(h):
    return m2s(h*60)

from collections import deque
class DelayBuffer(deque):
    def __init__(self, delay=30, startvalue=0):
        delay = int(delay)
        self.delay = delay
        super().__init__(startvalue for _ in range(delay + 1))

    def cycle(self, hd):
        self[0] = hd
        self.rotate(1)
        return self[0]

from types import FunctionType, MethodType
def printdir(o):
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o, a)
            if not isinstance(v, (FunctionType, MethodType)):
                print(a, v)

In [5]:
from officelib.xllib import *
xl = Excel()

wb = xl.ActiveWorkbook
wb = wb or xl.Workbooks.Add()

ws = xl.ActiveSheet
ws = ws or wb.Worksheets.Add()

cells = ws.Cells
cell_range = ws.Cells.Range

In [7]:
from math import sin, pi
class AgProcess():
    def __init__(self, m=1/2.76, b=0, delay=3, op=0, pamplitude=0.01, pperiod=30, ag_min=3):
        self.delay = DelayBuffer(delay, op).cycle
        self.m = m
        self.b = b
        self.x = self.t = 0
        self.pamplitude = pamplitude
        self.pperiod = pperiod
        self.pv = 0
        self.ag_min = ag_min
        
        
    def step(self, op):
        self.x += 1
        self.t += 1
        if self.x >= self.pperiod:
            self.x = 0
        m = self.m
        f = sin(self.x*2*pi/self.pperiod)*self.pamplitude + 1
        m *= f
        op = self.delay(op)
        self.pv = m*op + self.b
        if self.pv < self.ag_min:
            return 0
        return self.pv

In [8]:
def ag_sim(p,i,d,end,delay=3, op=0,pv=0,sp=8, amp=0.05, per=180):
    pid = PIDController(p,m2s(i),d,100,0, do_ifactor=True)
    pid.man_to_auto(pv, sp, op)
    proc = AgProcess(delay=delay, pamplitude=amp, pperiod=per)
    t = 0
    data = [(t, pv, op)]
    while True:
        t += 1
        if t < 15:
            pv = 0
        op = pid.step(pv, sp)
        data.append((t, pv, op))
        pv = proc.step(op)
        if t >= end: break
    return data

In [12]:
def paste(cells, data):
    cells.Range(cells(1,1), cells(1, 3)).Value = [("T", "PV", "OP")]
    topleft = cells(2,1)
    bottomright = topleft.Offset(len(data), len(data[0]))
    cells.Range(topleft, bottomright).Clear()
    cells.Range(topleft, bottomright).Value = data

def clear(cells, data):
    topleft = cells(2,1)
    bottomright = topleft.Offset(len(data), len(data[0]))
    cells.Range(topleft, bottomright).Clear()
    

In [14]:
clear(cells, data)
data = ag_sim(0.3, 0.02, 0, 180, 10, 0, 0, 8, 0.005*0, 50)
paste(cells, data)

In [15]:
data = ag_sim(0.1, 0.01, 0, 180, 4, 0, 0, 12, 0.005, 50)
paste(cells, data)